<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/chatbot%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

implemented by 李鲁鲁

- [x] 编写针对embedding的base64压缩，解压缩函数
- [x] 解压缩一个角色的文字包
- [X] clone Haruhi2 进行抽取
- [x] 抽取后保存成jsonl

TODO 米唯实

- [ ] 现在embedding抽取和加密似乎有点慢，看看有没有加速的可能性

请为我实现两个python函数 float_array_to_base64 和base64_to_float_array

前者输入一个list of float 输出一个str是将这个list编码为base64的字符串

后者输入这个字符串，解码输出list of float

这里我实现好了，放到了util

让我们重构程序，试图从Haruhi的util去进行调用

In [ ]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 719 (delta 12), reused 17 (delta 8), pack-reused 693
Receiving objects: 100% (719/719), 105.67 MiB | 36.42 MiB/s, done.
Resolving deltas: 100% (342/342), done.
/content/Haruhi-2-Dev


In [ ]:
!pip -q install transformers openai tiktoken langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s et

In [ ]:
from ChatHaruhi.utils import float_array_to_base64, base64_to_float_array

我们这里初始化Haruhi的类，但是实际上我们只使用embedding的功能

In [ ]:
# 以后有切换其他embedding的可能性

embedding_model = 'luotuo_openai'

In [ ]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi(role_name = 'haruhi', llm = 'debug', embedding = embedding_model)

print(chatbot.embedding)

<function luotuo_openai_embedding at 0x7dd5ea28a440>


实际上这里应该就已经可以用embedding给出句子的vec，我们测试一下

注意如果这里是英语的话 你需要openai的key

In [ ]:
test_vec = chatbot.embedding('我是一句话。')

print(test_vec[:4])

正在下载Luotuo-Bert


A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Luotuo-Bert下载完毕


[-0.014974536374211311, -0.01579006016254425, 0.018859559670090675, -0.0012475622352212667]


我们尝试一下编码再解码

In [ ]:
base64_str = float_array_to_base64(test_vec)
print(len(base64_str), base64_str[:200])

decoded_vec = base64_to_float_array(base64_str)
print(decoded_vec[:4])

8192 vHVXwryBWig8mn9duqOFPrw8Xyc7gydkvBl4q7xWqaI74YTZvES4TTxapEs8yHI5O9cX8rz6oAC8LlZ9vB86PTynbF06TxbRPKS5S7vgM2A8HzGAOHKvADxPGKK7618YvQpsADuaTww78yS0vCzSOru+PYy6gaEnPJCmkbw3yvK8FHK5O7YnS7vnNmy8il8uvKJPIbuq
[-0.014974536374211311, -0.01579006016254425, 0.018859559670090675, -0.0012475622352212667]


我们尝试来写一下打包函数, 在这之前我们先准备一下语料

我们在/content/中上传linghuchong

然后解压

In [ ]:
!unzip -q /content/linghuchong_text.zip -d /content/linghuchong
!unzip -q /content/yuebuqun_text.zip -d /content/yuebuqun

定义input_path

In [ ]:
texts_path = '/content/linghuchong/content/texts'

system_prompt = \
'''I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.

注意令狐冲对权威和规矩不在意,情愿浪迹江湖,不喜被束缚
令狐冲正义感强,重视信用,看不惯恶行。
'''

我们来尝试编写这个函数

In [ ]:
from ChatHaruhi.utils import package_role

In [ ]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

  5%|▌         | 51/937 [03:43<48:58,  3.32s/it]

In [ ]:
import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

In [ ]:
save_name = '/content/linghuchong.jsonl'

write_jsonl(datas, save_name)


我们再来弄一下岳掌门

In [ ]:
texts_path = '/content/yuebuqun/content/texts'

system_prompt = \
'''I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.

注意令狐冲对权威和规矩不在意,情愿浪迹江湖,不喜被束缚
令狐冲正义感强,重视信用,看不惯恶行。
'''

In [ ]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

In [ ]:
save_name = '/content/yuebuqun.jsonl'

write_jsonl(datas, save_name)
